In [1]:
#args
weights = 'yolov7.pt'
device = 'cuda'
cfg = './cfg/training/yolov7.yaml'
hyp = 'data/hyp.scratch.p5.yaml'
nc = 80

from models.yolo import Model
import torch
import yaml
from utils.torch_utils import intersect_dicts

c:\Users\luido\Anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Hyperparameters
with open(hyp) as f:
    hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps
pretrained = weights.endswith('.pt')
if pretrained:
    ckpt = torch.load(weights, map_location=device)  # load checkpoint
    det_model = Model(cfg or ckpt['model'].yaml, ch=3, nc=nc, anchors=hyp.get('anchors')).to(device)  # create
    exclude = ['anchor'] if (cfg or hyp.get('anchors')) else []  # exclude keys
    state_dict = ckpt['model'].float().state_dict()  # to FP32
    state_dict = intersect_dicts(state_dict, det_model.state_dict(), exclude=exclude)  # intersect
    det_model.load_state_dict(state_dict, strict=False)  # load

c:\Users\luido\Anaconda3\envs\torch-gpu\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
features = {}
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

In [5]:
det_model.model[50].register_forward_hook(get_features('backbone'))

In [6]:
dummy = torch.rand([1, 3, 384, 640]).to('cuda')
det_model(dummy)

[tensor([[[[[-1.12278e-01,  1.44947e+00, -3.56430e-01,  ..., -7.55253e+00, -8.62487e+00, -7.71362e+00],
            [ 3.01142e-01,  4.26621e-01,  8.82667e-01,  ..., -7.40901e+00, -8.09084e+00, -6.85711e+00],
            [ 7.56254e-01, -4.45289e-01,  5.75351e-02,  ..., -7.31858e+00, -8.42644e+00, -7.01480e+00],
            ...,
            [-1.99014e-01,  1.54507e-01,  2.14036e+00,  ..., -9.73159e+00, -8.60084e+00, -9.24294e+00],
            [ 1.14343e+00,  1.54357e-01,  4.44343e-01,  ..., -9.38422e+00, -8.62074e+00, -8.81272e+00],
            [-4.43269e-02, -1.66505e-01, -2.46456e-01,  ..., -8.51568e+00, -8.45737e+00, -8.67116e+00]],
 
           [[ 2.06981e-01,  9.49125e-02, -1.78426e-01,  ..., -7.84097e+00, -8.56259e+00, -8.52030e+00],
            [ 7.95512e-01,  4.51730e-01,  1.28995e+00,  ..., -8.29265e+00, -8.83446e+00, -9.44427e+00],
            [-6.93814e-01,  5.03537e-01,  1.77105e+00,  ..., -7.69283e+00, -8.78609e+00, -9.33325e+00],
            ...,
            [-2.51952e-01, 

In [8]:
features['backbone'].shape

torch.Size([1, 1024, 12, 20])